## Version Description:
 * In this version - we predict just the __R600 Hole Diameter__ (`R600_HD`)in TZ6 using [Bayesian Neural Network with MC DropOut](https://github.com/valyome/Neural-Networks-with-MC-Dropout/). 
 * Artificial data points synthesised from FDDN are used to train the ML model
 * The input features are `TZ6_Flow`,`MIXP`,`AMBP`,`AMBT` and the TZ6 CLR Restrictors.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.6g" % x)) 
from functools import reduce

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
plt.style.use('seaborn-whitegrid')

### Loading Data

In [4]:
input_features = ['TZ_Flow','MIXP','AMBP','AMBT','R610_HS1','R611_HS1','R612_HS1','R613_HS1']
usecols = ['HoV']+input_features+['R600_HD']

In [5]:
FDDN_df = pd.read_csv('../../data/Simulation_dataset.csv', usecols = usecols)[usecols]
FDDN_df.head()

,HoV,TZ_Flow,MIXP,AMBP,AMBT,R610_HS1,R611_HS1,R612_HS1,R613_HS1,R600_HD
0,FDDN_ASD0,720.292743,1800,102926,297.15,131,136,120,120,146
1,FDDN_ASD1,871.403000,2600,102426,299.15,131,136,120,120,146
2,FDDN_ASD2,985.488429,3400,103127,294.15,131,136,120,120,146
3,FDDN_ASD3,722.824643,1800,101825,295.15,131,136,120,120,146
4,FDDN_ASD4,875.039489,2600,100523,295.15,131,136,120,120,146


In [6]:
LTR_df = pd.read_csv('../../data/LTR_dataset.csv', usecols = usecols)[usecols]
LTR_df.head()

,HoV,TZ_Flow,MIXP,AMBP,AMBT,R610_HS1,R611_HS1,R612_HS1,R613_HS1,R600_HD
0,A1,887.134249,2600.0000,101401.6000,299.386667,131,136,120,120,148
1,A2,886.764050,2600.0000,101576.3000,298.448667,131,136,120,120,149
2,A3,926.224856,2606.1928,102136.6035,297.109024,131,136,114,120,152
3,A4,915.139474,2599.8998,103195.6642,295.060027,131,136,120,120,154
4,A5,891.635528,2600.0000,102856.2000,294.755833,145,153,130,130,148


### Train BNN with FDDN (Simulation) data:

In [7]:
# Extract pandas dataframe values to numpy array
train_data = FDDN_df[input_features+['R600_HD']].values
test_data  = LTR_df[input_features+['R600_HD']].values
print("Number of training samples:", len(train_data))
print("\nNumber of testing samples:", len(test_data))

# Separate arrays into input and output components (predictors and response variables)
x_train, y_train  = train_data[:,:-1] ,train_data[:,-1:]
x_test, y_test = test_data[:,:-1] ,test_data[:,-1:]

Number of training samples: 1080

Number of testing samples: 34


In [8]:
print('X_train Shape:',x_train.shape,'Y_Train Shape:', y_train.shape)
print('X_test Shape:',x_test.shape,'Y_Test Shape:', y_test.shape)

X_train Shape: (1080, 8) Y_Train Shape: (1080, 1)
X_test Shape: (34, 8) Y_Test Shape: (34, 1)


### Bayesian Neural Network using MC DropOut

In [9]:
import BNN_MonteCarlo_Dropout

Using TensorFlow backend.


In [10]:
%%time
# Train the BNN
bnn = BNN_MonteCarlo_Dropout.net(x_train, y_train, n_epochs=2000, n_hidden=[24,12], normalize=True)

Wall time: 19.8 s


In [11]:
%%time
# Test on ~25% of the data.
rmse, Yt_hat, MC_pred, predictive_variance = bnn.predict(x_test, y_test)

34/34 [==============================] - 0s 2ms/step
Standard rmse 1.498152
MC rmse 1.519688
test_ll -4.784226
Wall time: 571 ms


In [12]:
print('Shape of Predictions:',Yt_hat.shape)

Shape of Predictions: (1000, 1, 34, 1)


In [13]:
Yt_hat.shape[2]

34

In [14]:
y_preds = np.zeros((Yt_hat.shape[2], Yt_hat.shape[0])) # empty array to be populated
y_means = [] # save mean for each predicted point
y_std=[] # save standard dev for each predicted
y_mins = [] 
y_maxes = []

for j in range(Yt_hat.shape[2]):
    for i in range(Yt_hat.shape[0]):
        y_preds[j][i] = Yt_hat[i][0][j] # convert the predictions into a more readable format
    y_means.append(y_preds[j].mean()) # get the mean for each prediction
    y_std.append(y_preds[j].std()) # get the standard deviation
    y_mins.append(np.amin(y_preds[j],axis = 0)) # get the min value in the array
    y_maxes.append(np.amax(y_preds[j],axis = 0)) # get the min value in the array

In [15]:
print('Length of Y_Predictions:',len(y_preds))
print('Nr. of Predictions by BNN per test point:',len(y_preds[0]))

Length of Y_Predictions: 34
Nr. of Predictions by BNN per test point: 1000


In [16]:
print(y_means)

[147.7510035095215, 147.82408934020995, 152.32389736938475, 151.71386547851563, 147.95722723388673, 148.84778080749513, 148.14743395996095, 151.49361070251464, 151.12886154174805, 148.568497756958, 150.31457704162597, 152.85678744506836, 149.53960081481932, 148.1717176361084, 147.7981405181885, 149.22716041564942, 147.36155004882812, 147.22509704589845, 147.91888717651366, 147.4055891265869, 149.21772203063964, 147.8469543762207, 154.02765951538086, 153.03716255187987, 153.17583160400392, 147.98364697265626, 148.33501470947266, 152.46110702514648, 148.1733798828125, 149.43469929504394, 151.63404008483886, 154.05608343505858, 148.19926899719238, 148.19942529296875]


In [17]:
print(y_std)

[1.1188019731182601, 1.0430637606736033, 0.5832835305282, 0.6753795478915626, 0.9875512184370727, 0.9308466741178297, 1.0289461382357497, 0.5691830294826473, 0.59027528623263, 0.9427972228536936, 0.7528585675579436, 0.7046503320538889, 0.7894661924533118, 1.0210591619473537, 1.174526785394548, 0.835794220511231, 1.4962840104044302, 1.5190505322172114, 0.9999762820163506, 1.1691882145638994, 0.7686008187916766, 1.1371649582320815, 0.690346088131122, 0.8066034100874662, 0.6286982654636992, 1.4023405380756035, 0.9289010410030724, 0.619032355951114, 1.059841194472623, 0.7942826076600596, 0.6027248351929183, 0.7609186484949854, 1.2060302796514848, 1.0336761746199625]


In [18]:
print(y_mins)

[146.7043914794922, 146.75148010253906, 148.52662658691406, 148.62384033203125, 146.64593505859375, 147.05821228027344, 146.739501953125, 148.65643310546875, 148.5222930908203, 146.9248046875, 147.5689239501953, 148.743896484375, 147.69241333007812, 146.84117126464844, 146.47373962402344, 146.9357452392578, 146.23757934570312, 146.22845458984375, 146.80491638183594, 146.38604736328125, 147.119140625, 146.65924072265625, 149.5700225830078, 149.11758422851562, 148.847900390625, 146.5411376953125, 146.59861755371094, 148.5593719482422, 146.6545867919922, 147.08810424804688, 149.92359924316406, 149.62057495117188, 146.29067993164062, 146.4927215576172]


In [19]:
print(y_maxes)

[153.34384155273438, 153.53700256347656, 154.9805145263672, 154.45899963378906, 152.8528594970703, 153.478515625, 153.2069091796875, 154.1299591064453, 153.43756103515625, 153.61607360839844, 153.4003448486328, 155.74240112304688, 153.28895568847656, 153.98341369628906, 153.23362731933594, 154.3544464111328, 154.69329833984375, 154.05093383789062, 152.98350524902344, 153.16567993164062, 152.75877380371094, 153.82838439941406, 156.02435302734375, 156.5202178955078, 155.1416015625, 154.79408264160156, 152.58303833007812, 154.5173797607422, 153.3324432373047, 153.93548583984375, 153.91091918945312, 158.1661376953125, 153.3626708984375, 153.76260375976562]


In [20]:
y_true = LTR_df['R600_HD'].values
HoV_test_data = LTR_df['HoV'].values

In [21]:
print('No. of Test HoVs:',len(HoV_test_data),'\nHoVs of Test Points:',HoV_test_data)
print('Test Points-Org Y:', y_true)

No. of Test HoVs: 34 
HoVs of Test Points: ['A1' 'A2' 'A3' 'A4' 'A5' 'A6' 'C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'D1' 'D2' 'E1' 'E2' 'F1' 'F2' 'F3' 'H1' 'I1' 'J1' 'M1' 'M2' 'P1' 'Q1' 'Q2' 'S1' 'S2' 'S3' 'S4' 'S5' 'T1' 'T2']
Test Points-Org Y: [148 149 152 154 148 148 148 153 152 150 152 154 151 148 148 148 146 146 148 148 152 148 154 154 154 148 148 155 148 150 150 160 148 148]


### Plot BNN Predictions with Mean and Error Bars

In [22]:
from bokeh.models import HoverTool,ColumnDataSource,Label,Range1d
from bokeh.plotting import figure, show, output_file,output_notebook
from bokeh.models.markers import CircleX
output_notebook()

Loading BokehJS ...

In [23]:
# create the coordinates for the errorbars
y_xs = []
ys_std = []
y_ys = []

for x, y_mean,std,min_dia, max_dia in zip(HoV_test_data,y_means,y_std,y_mins,y_maxes):
    y_xs.append((x, x))
    ys_std.append((y_mean - std, y_mean + std))
    y_ys.append((y_mean - (y_mean - min_dia), y_mean + (max_dia - y_mean)))

In [24]:
# plot the points
f = figure(x_range=HoV_test_data, title='BNN MC DropOut - Dia Predictions with Confidence Interval', width=1400)
dot_hover_tooltips = HoverTool(names=['DiaPoints'],tooltips=[("HoV","@x"),("Dia", "@y mm")])
whisker_hover_tooltips = HoverTool(names=['Whiskers'],tooltips=[("Dia", "@y mm")])
multiline_hover_tooltips = HoverTool(names=['LinePlot'],tooltips=[("Dia", "$y mm")])

f.add_tools(dot_hover_tooltips,whisker_hover_tooltips,multiline_hover_tooltips)

f.xaxis.axis_label = 'Head of Versions'
f.yaxis.axis_label = 'Diameter Predictions (mm)'

f.multi_line(y_xs, ys_std, color='blue', line_width = 3, legend = 'Mean Prediction +/- Std.Dev')
f.multi_line(y_xs, y_ys, color='deepskyblue',name='LinePlot',legend = 'Mean Prediction +/- Min & Max Value')
f.diamond(HoV_test_data, y_true, color='red', size=12, line_alpha=0,name='DiaPoints', legend = 'True Dia')
f.circle(HoV_test_data, y_means, color='blue', size=9, line_alpha=0,name='DiaPoints', legend = 'Predicted Dia (Mean)')

# whiskers (almost-0 height rects simpler than segments)
for i in range(len(y_xs)):
    f.rect(y_xs[i], y_ys[i], 0.2, 0.001, line_color="deepskyblue",name='Whiskers')

f.y_range=Range1d(135, 175)
f.legend.location = "top_center"
f.legend.click_policy="hide"

output_file('plots/BNN_MCDropOut_v1_FDDN_LTR_Predictions.html', mode='inline')

In [25]:
show(f)